<a href="https://colab.research.google.com/github/satvik959/DL-2025-26/blob/main/WEEK5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub

import kagglehub
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

path = kagglehub.dataset_download("fedesoriano/stroke-prediction-dataset")

print("Dataset path:", path)

df = pd.read_csv(path + "/healthcare-dataset-stroke-data.csv")

print(df.head())

Using Colab cache for faster access to the 'stroke-prediction-dataset' dataset.
Dataset path: /kaggle/input/stroke-prediction-dataset
      id  gender   age  hypertension  heart_disease ever_married  \
0   9046    Male  67.0             0              1          Yes   
1  51676  Female  61.0             0              0          Yes   
2  31112    Male  80.0             0              1          Yes   
3  60182  Female  49.0             0              0          Yes   
4   1665  Female  79.0             1              0          Yes   

       work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0        Private          Urban             228.69  36.6  formerly smoked   
1  Self-employed          Rural             202.21   NaN     never smoked   
2        Private          Rural             105.92  32.5     never smoked   
3        Private          Urban             171.23  34.4           smokes   
4  Self-employed          Rural             174.12  24.0     never smoke

In [2]:
# Drop ID column (not useful for prediction)
df = df.drop(columns=["id"])

# Handle missing values (BMI has missing values)
df['bmi'] = df['bmi'].fillna(df['bmi'].mean())

# Convert categorical columns to numerical
df = pd.get_dummies(df, drop_first=True)

# Separate features and target
X = df.drop("stroke", axis=1)
y = df["stroke"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Feature scaling (VERY IMPORTANT for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
def build_l2_model(input_dim):
    model = models.Sequential([
        layers.Dense(64, activation='relu',
                     input_shape=(input_dim,),
                     kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(32, activation='relu',
                     kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(1, activation='sigmoid')   # Binary output
    ])
    return model

model_l2 = build_l2_model(X_train.shape[1])

model_l2.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
history = model_l2.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9043 - loss: 0.9946 - val_accuracy: 0.9548 - val_loss: 0.5546
Epoch 2/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9514 - loss: 0.5017 - val_accuracy: 0.9548 - val_loss: 0.3578
Epoch 3/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9537 - loss: 0.3325 - val_accuracy: 0.9548 - val_loss: 0.2679
Epoch 4/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9533 - loss: 0.2607 - val_accuracy: 0.9548 - val_loss: 0.2252
Epoch 5/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9535 - loss: 0.2189 - val_accuracy: 0.9548 - val_loss: 0.2004
Epoch 6/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9551 - loss: 0.1967 - val_accuracy: 0.9548 - val_loss: 0.1864
Epoch 7/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9577 - loss: 0.1760 - val_accuracy: 0.9548 - val_loss: 0.1782
Epoch 8/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9516 - loss: 0.1809 - val_accuracy: 0

In [5]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models, regularizers

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the input data
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# 🔹 FIX: Reshape 28x28 images into 784-length vectors
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define a simple model with L2 regularization
def build_l2_model():
    model = models.Sequential([
        layers.Dense(64, activation='relu',
                     input_shape=(784,),
                     kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(64, activation='relu',
                     kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(10, activation='softmax')
    ])
    return model

model_l2 = build_l2_model()

model_l2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model_l2.fit(
    x_train,
    y_train,
    epochs=10,
    validation_split=0.2,
    batch_size=32
)

# Evaluate the model
test_loss, test_acc = model_l2.evaluate(x_test, y_test)

print(f"Test accuracy: {test_acc:.4f}")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 8ms/step - accuracy: 0.8273 - loss: 1.2602 - val_accuracy: 0.9219 - val_loss: 0.5387
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.9143 - loss: 0.5477 - val_accuracy: 0.9171 - val_loss: 0.4988
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9273 - loss: 0.4771 - val_accuracy: 0.9362 - val_loss: 0.4393
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9338 - loss: 0.4381 - val_accuracy: 0.9477 - val_loss: 0.3854
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9379 - loss: 0.4086 - val_accuracy: 0.9407 - val_loss: 0.3851
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9428 - loss: 0.3821 - val_accuracy: 0.9518 - val_loss: 0.3589
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9441 - loss: 0.3674 - val_accuracy: 0.9514 - val_loss: 0.3356
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━

In [6]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# 🔹 Reshape to (samples, 28, 28, 1) for augmentation
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 🔹 Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)

datagen.fit(x_train)

# 🔹 Build Model (with L2 regularization)
def build_l2_model():
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(64, activation='relu',
                     kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(64, activation='relu',
                     kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(10, activation='softmax')
    ])
    return model

model_l2 = build_l2_model()

model_l2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 🔹 Train using augmented data
history = model_l2.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    epochs=10,
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 32
)

# 🔹 Evaluate model
test_loss, test_acc = model_l2.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 34s 17ms/step - accuracy: 0.6706 - loss: 1.6883 - val_accuracy: 0.9328 - val_loss: 0.6712
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - accuracy: 0.8652 - loss: 0.8325 - val_accuracy: 0.9449 - val_loss: 0.5597
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 32s 17ms/step - accuracy: 0.8882 - loss: 0.7176 - val_accuracy: 0.9401 - val_loss: 0.5224
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 17ms/step - accuracy: 0.8963 - loss: 0.6600 - val_accuracy: 0.9346 - val_loss: 0.5259
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.8988 - loss: 0.6217 - val_accuracy: 0.9468 - val_loss: 0.4582
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 32s 17ms/step - accuracy: 0.9022 - loss: 0.6003 - val_accuracy: 0.9494 - val_loss: 0.4417
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.9045 - loss: 0.5852 - val_accuracy: 0.9429 - val_loss: 0.4344
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 32s 17ms/step - accuracy: 0.9061 - loss: 0.57

In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# Load MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Flatten
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 🔹 Shared Layer
shared_dense = Dense(64, activation='relu')

# Two inputs
input1 = Input(shape=(784,))
input2 = Input(shape=(784,))

# Apply SAME layer (parameter sharing)
encoded1 = shared_dense(input1)
encoded2 = shared_dense(input2)

# Merge
merged = Concatenate()([encoded1, encoded2])
output = Dense(10, activation='softmax')(merged)

model = Model(inputs=[input1, input2], outputs=output)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train (using same data twice just for demo)
model.fit([x_train, x_train], y_train,
          epochs=5,
          batch_size=256,
          validation_split=0.2)

test_loss, test_acc = model.evaluate([x_test, x_test], y_test)
print("Test accuracy (Parameter Sharing):", test_acc)


Epoch 1/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.7134 - loss: 1.0128 - val_accuracy: 0.9275 - val_loss: 0.2674
Epoch 2/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9231 - loss: 0.2692 - val_accuracy: 0.9432 - val_loss: 0.2069
Epoch 3/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9425 - loss: 0.2014 - val_accuracy: 0.9503 - val_loss: 0.1783
Epoch 4/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9523 - loss: 0.1615 - val_accuracy: 0.9559 - val_loss: 0.1581
Epoch 5/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9609 - loss: 0.1365 - val_accuracy: 0.9601 - val_loss: 0.1434
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9549 - loss: 0.1548
Test accuracy (Parameter Sharing): 0.9595000147819519
